Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/notebooks/10/tmp/m_10_04.stan will be updated.



Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";


Inference for Stan model: m_10_04_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.2, 1.3, 1.3, 1.3) seconds, 5.1 seconds total
Sampling took (1.2, 1.2, 1.0, 1.2) seconds, 4.6 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%  N_Eff  N_Eff/s    R_hat
lp__             -261  5.2e-02     2.2   -265   -261   -259   1745      378  1.0e+00
accept_stat__    0.84  4.1e-03    0.21   0.35   0.92   1.00   2654      575  1.0e+00
stepsize__       0.37  1.8e-02   0.026   0.35   0.38   0.41    2.0     0.43  4.9e+13
treedepth__       3.2  9.6e-02    0.60    2.0    3.0    4.0     38      8.3  1.0e+00
n_leapfrog__       12  1.2e-01     6.1    7.0     11     23   2552      553  1.0e+00
divergent__     0.076  5.4e-03    0.27   0.00   0.00    1.0   2435      528  1.0e+00
energy__          266  7.7e-02     3.0    261    265    271   1574      341  1.0e+00
a[1]            -0.74  5.0e-03    0.27   -1.2  -0.

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpC"],
  :pooled => ["a.$i" for i in 1:7],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×16×4 Array{Float64,3}

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7
internals         = lp__, accept_stat__, stepsize__, treedepth__, n_leapfrog__, divergent__, energy__
parameters        = bp, bpC

parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8330 0.2626   0.0042 0.0059 1000
bpC -0.1239 0.2930   0.0046 0.0064 1000



Describe parameter draws

In [9]:
describe(chn2)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = bp, bpC

Empirical Posterior Estimates
─────────────────────────────────────────
parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8330 0.2626   0.0042 0.0059 1000
bpC -0.1239 0.2930   0.0046 0.0064 1000

Quantiles
─────────────────────────────────────────
parameters
      2.5%   25.0%   50.0%   75.0%  97.5%
 bp -0.0628  0.6559  0.8295 1.0099 1.7599
bpC -1.1784 -0.3224 -0.1283 0.0746 0.9457



Describe pooled parameter draws

In [10]:
describe(chn2, section=:pooled)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7

Empirical Posterior Estimates
───────────────────────────────────────────
pooled
      Mean    SD   Naive SE  MCSE   ESS
a.1 -0.7420 0.2709   0.0043 0.0045 1000
a.2 10.9208 5.3089   0.0839 0.1439 1000
a.3 -1.0457 0.2806   0.0044 0.0049 1000
a.4 -1.0482 0.2822   0.0045 0.0050 1000
a.5 -0.7354 0.2689   0.0043 0.0039 1000
a.6  0.2188 0.2682   0.0042 0.0043 1000
a.7  1.8245 0.4009   0.0063 0.0065 1000

Quantiles
───────────────────────────────────────────
pooled
      2.5%   25.0%   50.0%   75.0%   97.5% 
a.1 -1.7493 -0.9235 -0.7473 -0.5544  0.0624
a.2  2.2795  6.8233  9.8372 13.8009 35.0116
a.3 -2.0744 -1.2278 -1.0385 -0.8616 -0.1162
a.4 -2.1306 -1.2327 -1.0491 -0.8603 -0.0570
a.5 -1.6838 -0.9173 -0.7300 -0.5493  0.1899
a.6 -0.8816  0.0371  0.2154  0.3922  1.1233
a.7  0.5272  1.5422  1.8058  2.0846  3.4261



Make it a DataFrame

In [11]:
df = to_df(chn2, [:parameters, :pooled])

,bp,bpC,a.1,a.2,a.3,a.4,a.5,a.6,a.7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.73843,0.040043,-0.717293,13.0023,-1.38428,-0.92933,-0.92299,0.374405,1.82325
2,0.669553,0.3062,-0.765067,6.79637,-1.04071,-0.599035,-0.600536,-0.0524681,1.52596
3,0.0325078,0.495365,-0.698969,6.69542,-0.662615,-0.682252,-1.107,0.413951,2.92987
4,0.298365,0.410752,-0.343724,11.3958,-1.01913,-0.851404,-0.608346,0.082725,2.69649
5,0.379115,0.499755,-0.265046,5.84282,-0.801712,-0.680685,-0.767707,0.210337,2.70571
6,0.172134,0.286953,-0.661301,8.31654,-0.570806,-1.02282,-0.404429,-0.0279233,2.9147
7,0.134757,0.341624,-0.682555,15.6458,-0.424966,-0.873707,-0.774782,0.32715,2.74094
8,1.60938,-1.06161,-0.923555,16.9788,-0.920366,-1.12907,-1.19033,0.228309,2.4899
9,1.50762,-1.13733,-0.831134,18.448,-1.0369,-1.13457,-1.29547,0.259987,2.40661


End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*